## Import

In [37]:
import warnings

warnings.filterwarnings('ignore')

In [38]:
import pickle
import random as rnd
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import seaborn as sns
import sklearn
import statsmodels.api as sm
import xgboost as xgb
from matplotlib.font_manager import FontProperties
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis,
                                           QuadraticDiscriminantAnalysis)
from sklearn.ensemble import (AdaBoostRegressor, BaggingClassifier,
                              BaggingRegressor, ExtraTreesRegressor,
                              GradientBoostingRegressor, RandomForestRegressor,
                              VotingRegressor)
from sklearn.feature_selection import (RFE, SelectFromModel, SelectKBest,
                                       VarianceThreshold, chi2)
from sklearn.inspection import permutation_importance
from sklearn.linear_model import (ElasticNet, HuberRegressor, Lasso,
                                  LinearRegression, RANSACRegressor, Ridge,
                                  SGDRegressor, TheilSenRegressor)
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,
                             classification_report, confusion_matrix, log_loss,
                             precision_score, recall_score, roc_auc_score, mean_squared_error, r2_score)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     ShuffleSplit, StratifiedKFold,
                                     cross_val_score, cross_validate,
                                     train_test_split)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import (Binarizer, LabelEncoder, MinMaxScaler,
                                   Normalizer, OneHotEncoder,
                                   PolynomialFeatures, StandardScaler,
                                   normalize)
from sklearn.svm import SVR, LinearSVR
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from time import time


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

## Reading data:

datapath : "../data"

In [39]:
traindf = pd.read_csv('../data/train.csv', index_col="ID")
traindf.sample(2)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
37676,City Hotel,1,295,2016,April,18,28,0,2,2,0.0,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,Non Refund,1.0,NaN,0,Transient,5.715179,0,0,Canceled,2015-10-21
70875,City Hotel,1,482,2016,November,45,3,0,2,2,0.0,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,Non Refund,1.0,NaN,0,Transient,-19.781006,0,0,Canceled,2015-10-21


### Drop columns not in test data and Clean missing data
* Drop columns: `is_canceled`, `reservation_status`, `reservation_status_date`, where they aren't in test data

In [40]:
testCol = set(pd.read_csv('../data/test.csv', index_col="ID").columns)
removeCols = [col for col in traindf.columns if col not in testCol]
removeCols.remove("adr")
print(removeCols)

['is_canceled', 'reservation_status', 'reservation_status_date']


In [41]:
traindf = traindf.drop(removeCols, axis=1)
traindf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
49383,City Hotel,76,2016,July,27,2,0,1,2,0.0,0,BB,FRA,Online TA,TA/TO,0,0,0,D,D,0,No Deposit,9.0,NaN,0,Transient,132.306671,0,0
3646,City Hotel,124,2015,August,32,8,2,1,2,0.0,0,HB,ITA,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,17.0,NaN,0,Transient-Party,102.806695,0,0


### Missing data, processing agent, company, country data
The below code shows column name of missing data

In [42]:
tmp = pd.to_numeric((traindf.isnull().sum() / traindf.shape[0]).map('{:,.2f}'.format))
tmp[tmp > 0]

country    0.01
agent      0.14
company    0.94
dtype: float64

Probrabaly one booking with `company` is more unlikely to chacel, also the one with specific `agent`, so turn them to boolean

For `Country`, take the nlargest(10)

In [43]:
def Clean_Missing(traindf):
    traindf["children"] = traindf["children"].fillna(0)

    traindf["company"] = (~traindf["company"].isna()).astype("int64")
    traindf["agent"] = (~traindf["agent"].isna()).astype("int64")
    return traindf

    
def Train_Top_Country(traindf):
    TopCountrySet = set(traindf["country"].value_counts()[:10].index)
    traindf["country"] = traindf["country"].apply(lambda x:x if x in TopCountrySet else "Other")
    traindf = pd.get_dummies(traindf, prefix="", prefix_sep="", columns=["country"])
    return traindf, TopCountrySet

traindf = Clean_Missing(traindf)
traindf, TopCountrySet = Train_Top_Country(traindf)

In [44]:
traindf.sample(5)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,BEL,BRA,DEU,ESP,FRA,GBR,IRL,ITA,NLD,Other,PRT
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
16059,Resort Hotel,24,2015,October,44,26,7,15,1,0.0,0,BB,Corporate,Corporate,0,0,0,E,G,2,No Deposit,1,1,0,Transient-Party,43.419565,1,0,0,0,0,0,0,0,0,0,0,0,1
57608,Resort Hotel,25,2016,August,35,24,0,2,2,2.0,0,BB,Online TA,TA/TO,0,0,0,G,G,0,No Deposit,1,0,0,Transient,302.116509,1,1,0,0,0,0,0,0,0,0,0,1,0
53427,City Hotel,301,2016,July,31,30,1,1,1,0.0,0,HB,Offline TA/TO,TA/TO,0,0,0,A,A,1,No Deposit,1,0,0,Transient-Party,79.646775,0,0,0,0,1,0,0,0,0,0,0,0,0
78772,Resort Hotel,0,2017,January,1,1,1,0,2,0.0,0,BB,Direct,Direct,0,0,0,D,E,0,No Deposit,1,0,0,Transient,110.000000,0,0,0,0,0,0,0,1,0,0,0,0,0
20305,Resort Hotel,3,2015,December,50,10,0,3,2,0.0,0,BB,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,1,0,0,Transient,15.047242,0,0,0,0,0,0,0,0,0,0,0,0,1


### Feature Transformation

`children`: `children` + `babies`  
`stays_in_nights`: `stays_in_week_nights` + `stays_in_weekend_nights`    
`hotel`: Convert hotel as `Resort Hotel` to 0, `City Hotel` to 1  

Create new feature `is_same_room`: Check if `reserved_room_type` is same as `assigned_room_type`(If so 1, else 0)

drop: `arrival_date_year` -- While training, most of them are 0  
      `arrival_date_week_number` -- Too trivial  
      `arrival_date_day_of_month` -- Too trivial  

One hot encoding: `meal`  
One hot encoding: `reserved_room_type` and `assigned_room_type`  
One hot encoding: `arrival_date_month`  
One hot encoding: `market_segment` `distribution_channel` `deposit_type` `customer_type`

In [45]:
def Feature_Transform(traindf):
    traindf = pd.get_dummies(traindf, prefix="", prefix_sep="", columns=["arrival_date_month"])
    traindf["stays_in_nights"] = traindf["stays_in_week_nights"] + traindf["stays_in_weekend_nights"] 
    traindf["children"] = traindf["children"] + traindf["babies"]
    traindf = traindf.drop(["babies"], axis=1)
    traindf["hotel"] = traindf["hotel"].map({"Resort Hotel": 0, "City Hotel": 1})

    traindf = traindf.drop(columns=["arrival_date_year", "arrival_date_week_number", "arrival_date_day_of_month"])

    traindf["is_same_room"] = (traindf["reserved_room_type"] == traindf["assigned_room_type"]).map({True: 1, False: 0})
    traindf = pd.get_dummies(traindf, columns=["reserved_room_type", "assigned_room_type"], prefix=["reserved", "assigned"])

    traindf = pd.get_dummies(traindf, columns=["meal", "market_segment", "distribution_channel", "deposit_type", "customer_type"])
    return traindf

traindf = Feature_Transform(traindf)

Final Adjustment:

In [46]:
#traindf = traindf[traindf["adr"] > 10]

In [47]:
traindf.sample(2)

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,BEL,BRA,DEU,ESP,FRA,GBR,IRL,ITA,NLD,Other,PRT,April,August,December,February,January,July,June,March,May,November,October,September,stays_in_nights,is_same_room,reserved_A,reserved_B,reserved_C,reserved_D,reserved_E,reserved_F,reserved_G,reserved_H,reserved_L,reserved_P,assigned_A,assigned_B,assigned_C,assigned_D,assigned_E,assigned_F,assigned_G,assigned_H,assigned_I,assigned_K,assigned_L,assigned_P,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,market_segment_Undefined,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
64650,1,20,1,2,2,0.0,0,0,0,0,1,0,0,110.294927,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
74104,1,503,0,2,2,0.0,0,0,0,0,1,0,0,19.380407,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,2,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0


### Train Model and Evaluate best model performance:

Since I only know about regression, I only use SVR as my model, and save the result as 

In [48]:
new_train_x = traindf.drop("adr", axis=1).values
new_train_y = traindf["adr"].values

In [49]:
def HenHenModelSelection():
    regressors = [LinearRegression(),
                   Lasso(),
                   Ridge()
#                    LinearSVR()
                   ]

    from time import time
    for regr in regressors:

        before = time()
        print("="*30)
        print(regr.__class__.__name__)

        X_train, X_test, y_train, y_test = train_test_split(new_train_x, new_train_y, test_size=0.2)
        scaler = StandardScaler()
        scaler = scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        regr.fit(X_train, y_train)
        X_test = scaler.transform(X_test)
        y_pred = regr.predict(X_test)
    
        after = time()

        print(f"MSE: {mean_squared_error(y_test, y_pred)} \nR2_score: {r2_score(y_test, y_pred)}")
        print(f"花費{(after - before):.1f}s")
        
#for i in range(10):
#    HenHenModelSelection()

In [50]:
def AaronModelSelection():
    classifiers = [LinearRegression(),
                   ElasticNet(),
                   Lasso(),
                   Ridge(),
                   LinearSVR(),
                   HuberRegressor(),
                   RANSACRegressor(),
    #                TheilSenRegressor(),
                   AdaBoostRegressor(),
    #                BaggingRegressor(),
                   GradientBoostingRegressor(),
                   RandomForestRegressor(),
    #                DecisionTreeRegressor(),
                   RandomForestRegressor(),
                   KNeighborsRegressor(),
                   XGBRegressor()
                   ]

    from time import time
    from sklearn.pipeline import make_pipeline

    for clf in classifiers:
        before = time()
        print("="*30)
        print(clf.__class__.__name__)
        
        pipe = make_pipeline(StandardScaler(), clf)
        crossValScore = cross_validate(pipe, new_train_x, new_train_y, scoring=('r2', 'neg_mean_squared_error'), cv=5) 
        for metric, value in crossValScore.items():
            print(f"{metric}: {abs(np.mean(value)):.2f}")
        after = time()
        print(f"花費{(after - before):.1f}s")

    print("="*30)
    
#AaronModelSelection()

In [51]:
regr = GradientBoostingRegressor()
scaler = StandardScaler()
scaler = scaler.fit(new_train_x)
new_train_x = scaler.transform(new_train_x)
regr.fit(new_train_x, new_train_y)

GradientBoostingRegressor()

In [52]:
testdf = pd.read_csv('../data/test.csv', index_col="ID")

In [53]:
testdf.sample(2)

,hotel,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,required_car_parking_spaces,total_of_special_requests
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
94369,Resort Hotel,144,2017,April,16,16,1,0,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,242.0,NaN,0,Transient,0,0
107905,Resort Hotel,49,2017,June,25,24,1,1,3,0.0,0,HB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,15.0,NaN,0,Transient,0,0


In [54]:
testdf = Clean_Missing(testdf)
testdf = Feature_Transform(testdf)

testdf["country"] = testdf["country"].apply(lambda x:x if x in TopCountrySet else "Other")
testdf = pd.get_dummies(testdf, prefix="", prefix_sep="", columns=["country"])

In [55]:
testdf.sample(2)

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,April,August,July,June,May,stays_in_nights,is_same_room,reserved_A,reserved_B,reserved_C,reserved_D,reserved_E,reserved_F,reserved_G,reserved_H,reserved_P,assigned_A,assigned_B,assigned_C,assigned_D,assigned_E,assigned_F,assigned_G,assigned_H,assigned_I,assigned_K,assigned_P,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,BEL,BRA,DEU,ESP,FRA,GBR,IRL,ITA,NLD,Other,PRT
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
110508,1,120,2,3,3,0.0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,5,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
101198,1,420,2,1,2,0.0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,3,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1


In [56]:
for col in set(traindf.columns) ^ set(testdf.columns):
    testdf[col] = np.zeros(len(testdf))

In [57]:
testdf_x = testdf.drop(["adr"], axis=1)
testdf_x = scaler.transform(testdf_x)

In [58]:
y_pred = regr.predict(testdf_x)

In [59]:
testdf["adr"] = y_pred

In [60]:
testdf.sample(2)

,hotel,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,agent,company,days_in_waiting_list,required_car_parking_spaces,total_of_special_requests,April,August,July,June,May,stays_in_nights,is_same_room,reserved_A,reserved_B,reserved_C,reserved_D,reserved_E,reserved_F,reserved_G,reserved_H,reserved_P,assigned_A,assigned_B,assigned_C,assigned_D,assigned_E,assigned_F,assigned_G,assigned_H,assigned_I,assigned_K,assigned_P,meal_BB,meal_FB,meal_HB,meal_SC,meal_Undefined,market_segment_Aviation,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Corporate,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party,BEL,BRA,DEU,ESP,FRA,GBR,IRL,ITA,NLD,Other,PRT,November,adr,assigned_L,March,distribution_channel_Undefined,January,reserved_L,December,October,September,February,market_segment_Undefined
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
96250,1,111,1,3,2,0.0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,4,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.0,56.506565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101126,1,20,1,1,2,0.0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,2,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0.0,99.058684,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
TestAdr = pd.DataFrame(y_pred, columns=["adr"]) 

In [62]:
TestAdr.to_csv("..//data//TestAdr.csv", index=False)

## End of ducument